# Andmeteadus pilves: "Azure ML SDK" meetod

## Sissejuhatus

Selles märkmikus õpime, kuidas kasutada Azure ML SDK-d mudeli treenimiseks, juurutamiseks ja kasutamiseks Azure ML-i kaudu.

Eeltingimused:
1. Olete loonud Azure ML tööruumi.
2. Olete laadinud [Heart Failure andmestiku](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) Azure ML-i.
3. Olete selle märkmiku üles laadinud Azure ML Studio-sse.

Järgmised sammud on:

1. Loo eksperiment olemasolevas tööruumis.
2. Loo arvutusklaster.
3. Laadi andmestik.
4. Konfigureeri AutoML, kasutades AutoMLConfig-i.
5. Käivita AutoML eksperiment.
6. Uuri tulemusi ja leia parim mudel.
7. Registreeri parim mudel.
8. Juuruta parim mudel.
9. Kasuta lõpp-punkti.

## Azure Machine Learning SDK-spetsiifilised impordid


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## Tööruumi algatamine
Algata tööruumi objekt salvestatud konfiguratsioonist. Veendu, et konfiguratsioonifail asub aadressil .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Loo Azure ML eksperiment

Loome eksperimendi nimega 'aml-experiment' äsja algatatud tööruumis.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Loo arvutusklaster
AutoML-i käivitamiseks peate looma [arvutussihtkoha](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target).


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Andmed
Veenduge, et olete andmekogu Azure ML-i üles laadinud ja et võti on sama nimega kui andmekogu.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

## AutoML konfiguratsioon


In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## AutoML käivitus


In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

## Salvesta parim mudel


In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## Parima mudeli juurutamine

Käivitage järgmine kood, et juurutada parim mudel. Juurutamise olekut saate vaadata Azure ML portaalis. See samm võib võtta paar minutit.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Kasuta lõpp-punkti
Saad lisada sisendeid järgmisele sisendnäidisele.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
